In [0]:
dbutils.fs.rm("/FileStore/tables", True)

Out[2]: True

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, explode
import pyspark.sql.functions as f

In [0]:
# Extract
spark = SparkSession.builder.appName("ETL Pipeline").getOrCreate()
df = spark.read.text("/FileStore/tables/WordData.txt")

In [0]:
df.show()

+--------------------+
|               value|
+--------------------+
|This is a Japanes...|
|The team members ...|
|As the years pass...|
|If you don't like...|
|He was disappoint...|
|When he encounter...|
|Situps are a terr...|
|Toddlers feeding ...|
|Edith could decid...|
|Her daily goal wa...|
|Tomorrow will bri...|
|His son quipped t...|
|He wondered why a...|
|If my calculator ...|
|The hummingbird's...|
|He went on a whis...|
|This is the last ...|
|I come from a tri...|
|The delicious aro...|
|Weather is not tr...|
+--------------------+
only showing top 20 rows



In [0]:
# Transformation
df2 = df.withColumn("splitedData", f.split("value"," "))
df3 = df2.withColumn("words", explode("splitedData"))
wordsDF = df3.select("words")
wordCount = wordsDF.groupBy("words").count()

In [0]:
wordCount.show()

+-----------+-----+
|      words|count|
+-----------+-----+
|   Tomorrow|    4|
|         If|    8|
|      leave|    4|
|      corny|    4|
|        day|    4|
|preoccupied|    4|
|       even|    8|
|      crazy|    4|
|    bananas|    4|
|     priest|    4|
|        did|    4|
|    whether|    4|
|     Having|    4|
|        I'm|    4|
|      crime|    4|
|  surprised|    4|
|      James|    4|
|      could|    8|
|        buy|    4|
|        him|    8|
+-----------+-----+
only showing top 20 rows



In [0]:
driver = "org.postgresql.Driver"
url = "jdbc:postgresql://<connection-string>/"
table = "shorya_schema_pyspark.WordCount"
user = "postgres"
password = ""

wordCount.write.format("jdbc").option("driver", driver).option("url",url).option("dbtable", table).option("mode",
                                                                                                          "append").option("user"
                                                                                                                           ,user).option("password",password).save()